Set folder as before (shared task)


test_data.csv should be in data

In [51]:
folder = '/content/drive/MyDrive/digital text analysis/NLP/shared task'

In [52]:
def run_notebook(sub_path, keep_var):
  init_var = set(globals().keys())
  loc = f'{folder}/{sub_path}'
  %run "{loc}"
  cur_var = set(globals().keys())
  new_var = cur_var - init_var
  print('Deleting unnecessary variables...')
  for var in new_var:
    if var not in keep_var and var in globals():
      del globals()[var]

In [ ]:
import torch
import pandas as pd
import numpy as np
def test_data_transform(path):
    '''
    converts emo stab into neuroticism
    q1-3 into comments as list
    traits -> labels (dict)
    rows into dicts
    '''
    df = pd.read_csv(path)
    cols = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Humility']
    conversion = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    df[cols] = df[cols].apply(lambda col: col.map(conversion))

    #swap es to neuro
    conversion_es_neuro = {
        'low': 2,
        'medium': 1,
        'high': 0
    }
    df['Emotional stability'] = df['Emotional stability'].map(conversion_es_neuro)

    data = []
    for idx, row in df.iterrows():
        comments = [row[col] for col in ['Q1','Q2','Q3']]
        labels = {
            'openness': row['Openness'],
            'conscientiousness': row['Conscientiousness'],
            'extraversion': row['Extraversion'],
            'agreeableness': row['Agreeableness'],
            'neuroticism': row['Emotional stability'],
            'humility': row['Humility']
        }
        new_dict ={
            'id': row['id'],
            'comments': comments,
            'labels': labels
        }
        data.append(new_dict)
    return data
#TEST SET
try:
  path = f'{folder}/data/test_data.csv' 
except:
  path = '/content/drive/MyDrive/digital text analysis/NLP/shared task/data/val_data.csv' #if ran local change this to actual new path
test_data = test_data_transform(path)


In [54]:
#eval is all the way down in the notebook (CTRL-F train_mode == False)
train_mode = False
run_notebook(sub_path='subscripts/train/bert_multi_classhead.ipynb',keep_var={'all_predictions_list'})

Using device: cuda

Loading best weights for the final model.

--- Evaluating Final Model on Test Set ---

Starting predictions one by one:
Sample (original index 0):
  Input text (first part): ['When I was the chairman of a student organisation, there was some friction between two board members in the organisation. Conflict among peers is something I really struggle with, especially in situations as this one. Since I was the chairman, I had to remain neutral on the one hand, while on the other hand also making this conflict does not remain unresolved. Because of this friction, I had decided to conduct a small informal meeting in which both members could talk things out and I had to moderate their conversation. As a result of this conversation, both members could see eye to eye again and the fiction did not expand into something worse. This was one of the first times in which I had to moderate a discussion between two friends and both were relieved that I took the time to focus on thei

In [56]:
#FLIP PREDS FOR NEUROTICISM 4/5